# Gensim Topic Modeling, Take 2

When we started with topic modeling, we used the combined (merged) dataset as Jorge and Ilona had recommended. However, when we met with Sayan Mukherjee (6/28) (https://stat.duke.edu/people/sayan-mukherjee), he advised us to be wary of batch effects and to run LDA on the transcripts from the DVD and VA Datasets separately. Once we had our topics, we should look at the estimated term frequency within each selected topic vs overall term frequency. Should the estimated term frequency closely mirror the overall term frequency, that's not a good topic (as it's just telling us what's in the documents). For those topics that show more variation, look at the words that make up the topics and see how close these topics are.

Let's get started.

## Set Up

### Import Statements and Load Datasets

In [1]:
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords

import re
import numpy as np
import pandas as pd
from pprint import pprint

#gensim
import gensim
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel
from gensim.models import TfidfModel
from gensim.models import NormModel

#spacy for lemmatization
import spacy
import en_core_web_sm

#plotting tools
import pyLDAvis
import pyLDAvis.gensim
import matplotlib.pyplot as plt

import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)

from collections import Counter, defaultdict

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/preethiseshadri/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [2]:
#read in our transcripts
transcript_df = pd.read_csv("shared_data_merged.csv")

dvd = transcript_df[transcript_df["Study"] == "DVD"]

va = transcript_df[transcript_df["Study"] == "VA"]

In [3]:
dvd = dvd[(pd.isnull(dvd.Convo_1) == False) & (pd.isnull(dvd.txgot_binary) == False)]

va = va[(pd.isnull(va.Convo_1) == False) & (pd.isnull(va.txgot_binary) == False)]

### Methods

In [4]:
def sent_to_words(sentences):
    for sentence in sentences:
        yield(gensim.utils.simple_preprocess(str(sentence), deacc=True))

def remove_stopwords(texts):
    return [[word for word in simple_preprocess(str(doc)) if word not in stopWords] for doc in texts]

def make_bigrams_dvd(texts):
    return [bigram_mod_dvd[doc] for doc in texts]

def make_trigrams_dvd(texts):
    return [trigram_mod_dvd[bigram_mod_dvd[doc]] for doc in texts]

def make_bigrams_va(texts):
    return [bigram_mod_va[doc] for doc in texts]

def make_trigrams_dvd(texts):
    return [trigram_mod_va[bigram_mod_va[doc]] for doc in texts]

nlp = en_core_web_sm.load(disable=['parser', 'ner'])

def lemmatization(texts, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV']):
    """https://spacy.io/api/annotation"""
    texts_out = []
    for sent in texts:
        doc = nlp(" ".join(sent)) 
        texts_out.append([token.lemma_ for token in doc if token.pos_ in allowed_postags])
    return texts_out        

def compute_coherence_values(dictionary, corpus, texts, limit, start, step):
    """
    Compute c_v coherence for various number of topics

    Parameters:
    ----------
    dictionary : Gensim dictionary
    corpus : Gensim corpus
    texts : List of input texts
    limit : Max num of topics

    Returns:
    -------
    model_list : List of LDA topic models
    coherence_values : Coherence values corresponding to the LDA model with respective number of topics
    """
    coherence_values = []
    model_list = []
    for num_topics in range(start, limit, step):
        model = gensim.models.wrappers.LdaMallet(mallet_path, corpus=corpus, num_topics=num_topics, id2word=id2word)
        model_list.append(model)
        coherencemodel = CoherenceModel(model=model, texts=texts, dictionary=dictionary, coherence='c_v')
        coherence_values.append(coherencemodel.get_coherence())

    return model_list, coherence_values

def format_topics_sentences(ldamodel, corpus, texts):
    # Init output
    sent_topics_df = pd.DataFrame()

    # Get main topic in each document
    for i, row in enumerate(ldamodel[corpus]):
        row = sorted(row, key=lambda x: (x[1]), reverse=True)
        # Get the Dominant topic, Perc Contribution and Keywords for each document
        for j, (topic_num, prop_topic) in enumerate(row):
            if j == 0:  # => dominant topic
                wp = ldamodel.show_topic(topic_num)
                topic_keywords = ", ".join([word for word, prop in wp])
                sent_topics_df = sent_topics_df.append(pd.Series([int(topic_num), round(prop_topic,4), topic_keywords]), ignore_index=True)
            else:
                break
    sent_topics_df.columns = ['Dominant_Topic', 'Perc_Contribution', 'Topic_Keywords']

    # Add original text to the end of the output
    contents = pd.Series(texts)
    sent_topics_df = pd.concat([sent_topics_df, contents], axis=1)
    return(sent_topics_df)

### Stop Words

In [23]:
stopWords = ["'", 'a', 'aa', 'aaah', 'aah', 'ab', 'about', 'above', 'african', 'after', 'again', 'against', 'ah', 'ahh', 'ahhh', 'ahhhh', 'ahhm', 'ain', 'aint', 'alabama', 'alaska', 'all', 'alot', 'alright', 'alrighty', 'also', 'am', 'an', 'anand', 'and', 'andand', 'any', 'anyone', 'are', 'aren', 'arent', 'as', 'at', 'ay', 'b', 'be', 'because', 'been', 'before', 'being', 'below', 'between', 'both', 'but', 'by', 'bye', 'c', 'california', 'came', 'can', 'cant', 'clean', 'communist', 'costa_rica', 'could', 'couldn', 'couldnt', 'cuz', 'd', 'de', 'did', 'didn', 'didnt', 'do', 'doc', 'does', 'doesn', 'doesnt', 'doin', 'doing', 'dokey', 'don', 'dont', 'down', 'during', 'e', 'each', 'eek', 'eh', 'em', 'er', 'et', 'etc', 'europe', 'f', 'few', 'florida', 'for', 'from', 'further', 'g', 'ga', 'gal', 'gee', 'geez', 'germany', 'get', 'go', 'goin', 'going', 'gonna', 'gosh', 'got', 'gotta', 'greek', 'gu', 'h', 'ha', 'had', 'hadn', 'hadnt', 'has', 'hasn', 'hasnt', 'have', 'haven', 'havent', 'having', 'he', 'hed', 'heh', 'hell', 'hello', 'henry', 'her', 'here', 'hers', 'herself', 'hes', 'hey', 'hi', 'him', 'himself', 'his', 'hm', 'hmm', 'hmmm', 'hodgkins', 'how', 'hows', 'huh', 'hum', 'i', 'id', 'if', 'ifif', 'ii', 'iii', 'ill', 'im', 'imrt', 'in', 'inaudible', 'indecipherable', 'indianapolis', 'interview_length', 'into', 'is', 'isis', 'isn', 'isnt', 'it', 'itd', 'itit', 'itll', 'its', 'itself', 'ive', 'j', 'jeez', 'just', 'k', 'kay', 'kinda', 'l', 'laughs', 'le', 'leastno', 'legend', 'let', 'lets', 'like', 'll', 'look', 'lot', 'm', 'ma', 'maam', 'md', 'mdmd', 'me', 'mhm', 'mhmm', 'mhmmm', 'michigan', 'mightn', 'mightnt', 'mightve', 'mkay', 'mm', 'mmhm', 'mmhmm', 'mmhmmm', 'mmkay', 'mmm', 'mmmhmm', 'mmmhmmm', 'mmmm', 'mmmmm', 'more', 'most', 'mustn', 'mustnt', 'mustve', 'my', 'myself', 'n', 'na', 'nah', 'nahuh', 'nd', 'ne', 'needn', 'neednt', 'nn', 'no', 'nooh', 'noooo', 'nope', 'nor', 'not', 'now', 'o', 'of', 'off', 'oh', 'ohh', 'ohhh', 'ohhhohohohoh', 'ohio', 'ok', 'okay', 'okey', 'on', 'once', 'only', 'oooh', 'or', 'oth', 'other', 'othumhmm', 'oughta', 'our', 'ours', 'ourselves', 'out', 'over', 'ow', 'own', 'p', 'patient', 'phi', 'physician', 'potter', 'pt', 'pt/so', 'q', 'r', 'rd', 're', 'right', 'ro', 's', 'said', 'same', 'say', 'see', 'shan', 'shant', 'she', 'shell', 'shes', 'should', 'shouldn', 'shouldnt', 'shouldve', 'significant_other', 'so', 'some', 'sorta', 'sounds', 'st', 'stuff', 'such', 'swedish', 't', 'taiwan', 'taiwanese', 'th', 'than', 'that', 'thatd', 'thatll', 'thats', 'thatsthat', 'the', 'their', 'theirs', 'them', 'themselves', 'then', 'there', 'thered', 'thereof', 'theres', 'thereve', 'these', 'thethe', 'thew', 'they', 'theyll', 'theyre', 'theyve', 'thing', 'things', 'this', 'those', 'through', 'ti', 'to', 'too', 'tthe', 'u', 'uh', 'uhh', 'uhhhhh', 'uhhm', 'uhhmm', 'uhhuh', 'uhm', 'uhmhmm', 'uhmhmmm', 'uhmmm', 'uhoh', 'uhum', 'um', 'umhmm', 'umhmmm', 'umm', 'ummm', 'ummmm', 'un', 'under', 'unhunh', 'until', 'up', 'us', 'uuh', 'v', 've', 'very', 'vietnam', 'virginia', 'w', 'walsh', 'wanna', 'was', 'washington', 'wasn', 'wasnt', 'we', 'wed', 'well', 'went', 'were', 'weren', 'werent', 'weve', 'wewe', 'what', 'whatd', 'whatev', 'whatever', 'whatnot', 'whats', 'when', 'where', 'wheres', 'whew', 'which', 'while', 'who', 'whoa', 'whom', 'whos', 'why', 'will', 'with', 'won', 'wont', 'would', 'wouldn', 'wouldnt', 'x', 'y', 'ya', 'yada', 'yah', 'yall', 'yea', 'yeah', 'yep', 'yepvery', 'yer', 'yeyeah', 'you', 'youd', 'youl', 'youll', 'your', 'youre', 'yours', 'yourself', 'yourselves', 'youve', 'youyou', 'yup', 'z']

### Pre Processing

In [6]:
data_dvd = dvd["Convo_1"]
data_va = va["Convo_1"]

print("We have", len(data_dvd), "conversations from DVD.")
print("We have", len(data_va), "conversations from VA.")

We have 140 conversations from DVD.
We have 216 conversations from VA.


In [7]:
data_words_dvd = list(sent_to_words(data_dvd))
data_words_va = list(sent_to_words(data_va))

##### Bigrams and Trigrams

In [8]:
# Build the bigram and trigram models
bigram_dvd = gensim.models.Phrases(data_words_dvd, min_count=2, threshold=100) 
trigram_dvd = gensim.models.Phrases(bigram_dvd[data_words_dvd], threshold=100)  

bigram_va = gensim.models.Phrases(data_words_va, min_count=2, threshold=100) 
trigram_va = gensim.models.Phrases(bigram_va[data_words_va], threshold=100)  

# Faster way to get a sentence clubbed as a trigram/bigram
bigram_mod_dvd = gensim.models.phrases.Phraser(bigram_dvd)
trigram_mod_dvd = gensim.models.phrases.Phraser(trigram_dvd)

bigram_mod_va = gensim.models.phrases.Phraser(bigram_va)
trigram_mod_va = gensim.models.phrases.Phraser(trigram_va)

/usr/local/Cellar/python3/3.6.2/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/gensim/models/phrases.py:494: UserWarning: For a faster implementation, use the gensim.models.phrases.Phraser class
  warnings.warn("For a faster implementation, use the gensim.models.phrases.Phraser class")


##### Remove Stop Words and Lemmatize

In [24]:
#remove stop words
data_words_nostops_dvd = remove_stopwords(data_words_dvd)
data_words_nostops_va = remove_stopwords(data_words_va)

#form bigrams
data_words_bigrams_dvd = make_bigrams_dvd(data_words_nostops_dvd)
data_words_bigrams_va = make_bigrams_va(data_words_nostops_va)

# Do lemmatization keeping only noun, adj, vb, adv
data_lemmatized_dvd = lemmatization(data_words_bigrams_dvd, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV'])
data_lemmatized_va = lemmatization(data_words_bigrams_va, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV'])

# Only keep conversations with at least 10 words
data_lemmatized_dvd = [convo for convo in data_lemmatized_dvd if len(convo) > 10]
data_lemmatized_va = [convo for convo in data_lemmatized_va if len(convo) > 10]

data_lemmatized_dvd = [[word for word in convo if word not in stopWords] for convo in data_lemmatized_dvd]
data_lemmatized_va = [[word for word in convo if word not in stopWords] for convo in data_lemmatized_va]

### Create Corpus

In [25]:
id2word_dvd = corpora.Dictionary(data_lemmatized_dvd)
id2word_va = corpora.Dictionary(data_lemmatized_va)

#Filter dictionary
#keep this in, leave it out?
id2word_dvd.filter_extremes(no_below = 0.3, no_above = 0.60, keep_n = 5000, keep_tokens = None)
id2word_va.filter_extremes(no_below = 0.3, no_above = 0.60, keep_n = 5000, keep_tokens = None)

#creates corpus
texts_dvd = data_lemmatized_dvd
texts_va = data_lemmatized_va

#tf-idf
corpus_dvd = [id2word_dvd.doc2bow(text) for text in texts_dvd]
corpus_va = [id2word_va.doc2bow(text) for text in texts_va]

corp_dvd = corpus_dvd
corp_va = corpus_va

tfidf_dvd = gensim.models.TfidfModel(corp_dvd, id2word=id2word_dvd)
tfidf_va = gensim.models.TfidfModel(corp_va, id2word=id2word_va)

## General LDA Model

In [26]:
#dvd
lda_model_dvd = gensim.models.ldamodel.LdaModel(corpus=corp_dvd,
                                           id2word=id2word_dvd,
                                           num_topics=12, 
                                           random_state=100,
                                           update_every=3,
                                           chunksize=20,
                                           passes=10,
                                           alpha='auto',
                                           per_word_topics=True)
#va
lda_model_va = gensim.models.ldamodel.LdaModel(corpus=corp_va,
                                           id2word=id2word_va,
                                           num_topics=10, 
                                           random_state=100,
                                           update_every=3,
                                           chunksize=20,
                                           passes=10,
                                           alpha='auto',
                                           per_word_topics=True)

### Topic Words and Weights

In [27]:
#view these topics
print("DVD Topics:")
pprint(lda_model_dvd.print_topics())
doc_lda_dvd = lda_model_dvd[corp_dvd]
print()
print("VA Topics:")
pprint(lda_model_va.print_topics())
doc_lda_va = lda_model_va[corp_va]

DVD Topics:
[(0,
  '0.009*"walk" + 0.008*"seed" + 0.007*"brother" + 0.006*"medicine" + '
  '0.006*"table" + 0.006*"dad" + 0.005*"blood_pressure" + 0.005*"smoke" + '
  '0.005*"heart_attack" + 0.004*"medical"'),
 (1,
  '0.007*"rectum" + 0.006*"active_surveillance" + 0.006*"cell" + 0.005*"gland" '
  '+ 0.005*"situation" + 0.005*"main" + 0.005*"seed" + 0.004*"medication" + '
  '0.004*"half" + 0.004*"morning"'),
 (2,
  '0.006*"impair" + 0.003*"chemo" + 0.003*"aww" + 0.003*"engage" + '
  '0.002*"mindset" + 0.002*"technical" + 0.001*"transmit" + '
  '0.001*"granddaughter" + 0.001*"toxic" + 0.001*"nevertheless"'),
 (3,
  '0.000*"seed" + 0.000*"active_surveillance" + 0.000*"group" + 0.000*"half" + '
  '0.000*"rectum" + 0.000*"sample" + 0.000*"gland" + 0.000*"intermediate" + '
  '0.000*"cell" + 0.000*"dose"'),
 (4,
  '0.003*"fixable" + 0.001*"giving" + 0.001*"win" + 0.000*"comparing" + '
  '0.000*"propensity" + 0.000*"cyberknife" + 0.000*"cell" + '
  '0.000*"active_surveillance" + 0.000*"dose" +

DVD dominant topics 
(0.3, 0.6, 5000 terms, 12 topics)
+ 0: general medical (walk, brother, dad, medicine, blood pressure, smoke, medical, heart attack), 1: surveillance (active_surveillance), 6: radiation (seed, external beam, dose), 8: surgery (erectile_dysfunction, erectile, robot)

VA dominant topics (topics are not as clear): 
(0.7, 5000 terms, 12 topics)
+ 5: general cancer/potential side effects (medication, urinary, leakage, therapy, grade, spread), 6: surveillance (core, grade, therpy, surveillance, keep), 8: surgery (robotic, open, procedure, leak, incision, catheter, robot), 9: radiation (seed, nerve?, tissue?)

(0.2, 0.55, 7000 terms, 12 topics), (0.2, 0.6, 5000 terms, 12 topics)
+ 1: surveillance (active surveillance), 6: radiation (dose, seed, externl beam), 8: surgery (erectile dysfunction, erectile, robot)

### Perplexity and Coherence

### Visualizations

In [29]:
print("VA:")
pyLDAvis.enable_notebook()
vis_va = pyLDAvis.gensim.prepare(lda_model_va, corp_va, id2word_va)
vis_va

VA:


/usr/local/Cellar/python3/3.6.2/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/pyLDAvis/_prepare.py:257: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=True'.

To retain the current behavior and silence the warning, pass sort=False

  return pd.concat([default_term_info] + list(topic_dfs))


PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
1      0.196838  0.059854       1        1  48.895489
6      0.207689  0.067558       2        1  40.392296
8      0.163106 -0.132943       3        1   6.929753
5     -0.050849  0.086561       4        1   1.374595
7     -0.036659 -0.122521       5        1   1.240884
0     -0.069253  0.002679       6        1   0.817408
2     -0.113040  0.012681       7        1   0.164227
9     -0.106992  0.009226       8        1   0.112576
3     -0.096001  0.008273       9        1   0.039146
4     -0.094839  0.008631      10        1   0.033623, topic_info=     Category        Freq                  Term       Total  loglift  logprob
term                                                                         
45    Default  496.000000                 grade  496.000000  30.0000  30.0000
110   Default  303.000000             typically  303.000000  29.0000  29.0000
370   Default  289.000000                tissue  289.000000  28.0000  28.0000
367   Default  166.000000                  tend  166.000000  27.0000  27.0000
222   Default  331.000000          incontinence  331.000000  26.0000  26.0000
4975  Default  128.000000           doctor_name  128.000000  25.0000  25.0000
8     Default  336.000000                  area  336.000000  24.0000  24.0000
106   Default  358.000000                  term  358.000000  23.0000  23.0000
706   Default  257.000000                volume  257.000000  22.0000  22.0000
28    Default  364.000000                   die  364.000000  21.0000  21.0000
277   Default  224.000000            oncologist  224.000000  20.0000  20.0000
557   Default  209.000000          surveillance  209.000000  19.0000  19.0000
188   Default  236.000000  erectile_dysfunction  236.000000  18.0000  18.0000
417   Default  214.000000               benefit  214.000000  17.0000  17.0000
119   Default  311.000000                affect  311.000000  16.0000  16.0000
329   Default  206.000000              schedule  206.000000  15.0000  15.0000
255   Default  287.000000            medication  287.000000  14.0000  14.0000
87    Default  218.000000                repeat  218.000000  13.0000  13.0000
806   Default  283.000000            understand  283.000000  12.0000  12.0000
616   Default  134.000000             everybody  134.000000  11.0000  11.0000
766   Default  148.000000                middle  148.000000  10.0000  10.0000
693   Default  171.000000               surgeon  171.000000   9.0000   9.0000
491   Default  258.000000                  meet  258.000000   8.0000   8.0000
75    Default  237.000000              positive  237.000000   7.0000   7.0000
32    Default  207.000000                enough  207.000000   6.0000   6.0000
425   Default   94.000000               brother   94.000000   5.0000   5.0000
241   Default  268.000000                  life  268.000000   4.0000   4.0000
49    Default  185.000000               healthy  185.000000   3.0000   3.0000
184   Default   95.000000                   dry   95.000000   2.0000   2.0000
18    Default  181.000000             certainly  181.000000   1.0000   1.0000
...       ...         ...                   ...         ...      ...      ...
111   Topic10    0.042154                 upbut    0.672886   5.2275  -6.6166
41    Topic10    0.042149          fourandahalf    0.672897   5.2273  -6.6167
77    Topic10    0.206944                proton    3.327911   5.2201  -5.0255
6     Topic10    0.133337            appearance    3.405430   4.7575  -5.4650
2596  Topic10    0.006083                 vodka    0.678631   3.2832  -8.5524
2595  Topic10    0.006083                 uhhhh    0.678690   3.2831  -8.5524
2395  Topic10    0.006083           fundamental    0.678697   3.2831  -8.5524
2417  Topic10    0.006083            prostetomy    0.678697   3.2831  -8.5524
2420  Topic10    0.006083            reflection    0.678698   3.2831  -8.5524
2382  Topic10    0.006083        

In [28]:
print("DVD:")
pyLDAvis.enable_notebook()
vis_dvd = pyLDAvis.gensim.prepare(lda_model_dvd, corp_dvd, id2word_dvd)
vis_dvd

DVD:


/usr/local/Cellar/python3/3.6.2/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/pyLDAvis/_prepare.py:257: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=True'.

To retain the current behavior and silence the warning, pass sort=False

  return pd.concat([default_term_info] + list(topic_dfs))


PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
8     -0.069477 -0.060179       1        1  33.601299
6     -0.181714  0.015098       2        1  18.754356
1     -0.139978 -0.021149       3        1  18.536728
7     -0.137779  0.093313       4        1  10.513648
10    -0.147078  0.141389       5        1  10.354227
0     -0.098831 -0.223799       6        1   8.080855
9      0.149339  0.008635       7        1   0.129541
2      0.129219  0.010224       8        1   0.021448
4      0.125020  0.009220       9        1   0.004488
11     0.123652  0.009070      10        1   0.001172
3      0.123749  0.009096      11        1   0.001138
5      0.123880  0.009082      12        1   0.001104, topic_info=     Category        Freq                  Term       Total  loglift  logprob
term                                                                         
1281  Default  234.000000          surveillance  234.000000  30.0000  30.0000
4     Default  394.000000   active_surveillance  394.000000  29.0000  29.0000
105   Default  144.000000                   dad  144.000000  28.0000  28.0000
625   Default  440.000000                 gland  440.000000  27.0000  27.0000
129   Default  299.000000                  dose  299.000000  26.0000  26.0000
410   Default  296.000000                sample  296.000000  25.0000  25.0000
422   Default  368.000000                  seed  368.000000  24.0000  24.0000
27    Default  164.000000              approach  164.000000  23.0000  23.0000
259   Default  216.000000            likelihood  216.000000  22.0000  22.0000
467   Default  348.000000                 study  348.000000  21.0000  21.0000
429   Default  289.000000       sexual_function  289.000000  20.0000  20.0000
417   Default   54.000000                school   54.000000  19.0000  19.0000
1130  Default  131.000000                 robot  131.000000  18.0000  18.0000
521   Default  153.000000                  walk  153.000000  17.0000  17.0000
156   Default  103.000000             extensive  103.000000  16.0000  16.0000
268   Default  198.000000            lymph_node  198.000000  15.0000  15.0000
626   Default  215.000000                 grade  215.000000  14.0000  14.0000
1842  Default  123.000000         brachytherapy  123.000000  13.0000  13.0000
144   Default  225.000000  erectile_dysfunction  225.000000  12.0000  12.0000
284   Default  212.000000              medicine  212.000000  11.0000  11.0000
221   Default  159.000000               implant  159.000000  10.0000  10.0000
104   Default  233.000000                  cure  233.000000   9.0000   9.0000
157   Default  233.000000         external_beam  233.000000   8.0000   8.0000
853   Default  104.000000                   fix  104.000000   7.0000   7.0000
743   Default   96.000000        small_incision   96.000000   6.0000   6.0000
391   Default  362.000000                rectum  362.000000   5.0000   5.0000
196   Default  200.000000                 group  200.000000   4.0000   4.0000
121   Default  259.000000            difference  259.000000   3.0000   3.0000
961   Default  135.000000             situation  135.000000   2.0000   2.0000
591   Default  130.000000                   dry  130.000000   1.0000   1.0000
...       ...         ...                   ...         ...      ...      ...
4     Topic12    0.000212   active_surveillance  394.790375  -3.0233  -8.5070
410   Topic12    0.000212                sample  296.169708  -2.7367  -8.5078
221   Topic12    0.000212               implant  159.913101  -2.1212  -8.5086
121   Topic12    0.000212            difference  259.157593  -2.6046  -8.5091
129   Topic12    0.000211                  dose  299.125305  -2.7488  -8.5100
467   Topic12    0.000211                 study  348.041595  -2.9004  -8.5101
69    Topic12    0.000211                  cell  331.217346  -2.8513  -8.5106
199   Topic12    0.000211                  half  221.942154  -2.4511  -8.5107
422

In [ ]:
print("DVD:")
print('\nPerplexity: ', lda_model_dvd.log_perplexity(corp_dvd)) #a measure of how good the model is

coherence_model_lda_dvd = CoherenceModel(model=lda_model_dvd, texts=data_lemmatized_dvd, dictionary=id2word_dvd, coherence='c_v')
coherence_lda_dvd = coherence_model_lda_dvd.get_coherence()
print('\nCoherence Score: ', coherence_lda_dvd)

print()
print("VA:")
print('\nPerplexity: ', lda_model_va.log_perplexity(corp_va)) #a measure of how good the model is

coherence_model_lda_va = CoherenceModel(model=lda_model_va, texts=data_lemmatized_va, dictionary=id2word_va, coherence='c_v')
coherence_lda_va = coherence_model_lda_va.get_coherence()
print('\nCoherence Score: ', coherence_lda_va)

# Dominant Topic Distributions Over Documents

In [30]:
distributions_va = lda_model_va[corpus_va]
distributions_dvd = lda_model_dvd[corpus_dvd]

In [31]:
# see which topics are the dominant topics
dominant_topics_va = defaultdict(lambda: 0)
for en, row in enumerate(distributions_va):
    for val in row[0]:
        if val[1] >= 0.05:
            dominant_topics_va[val[0]] += 1
dominant_topics_va

defaultdict(<function __main__.<lambda>()>,
            {1: 198, 4: 1, 6: 194, 8: 53, 0: 11, 5: 16, 7: 16, 2: 3, 9: 1})

In [32]:
# see which topics are the dominant topics
dominant_topics_dvd = defaultdict(lambda: 0)
for en, row in enumerate(distributions_dvd):
    for val in row[0]:
        if val[1] >= 0.05:
            dominant_topics_dvd[val[0]] += 1
dominant_topics_dvd

defaultdict(<function __main__.<lambda>()>,
            {1: 74, 6: 54, 7: 60, 8: 124, 0: 65, 10: 45, 9: 2, 2: 1})

In [33]:
# topics 0, 1, 7, and 8 are the most prominent topics in VA
dvd_length = len(corpus_dvd)
topic0 = [0] * dvd_length
topic1 = [0] * dvd_length
topic6 = [0] * dvd_length
topic8 = [0] * dvd_length

In [34]:
# store the topic percentage values for the relevant topics
for en, row in enumerate(distributions_dvd):
    topics = row[0]
    for topic in topics:
        if topic[0] == 0:
            topic0[en] = topic[1]
        elif topic[0] == 1:
            topic1[en] = topic[1]
        elif topic[0] == 6:
            topic6[en] = topic[1]
        elif topic[0] == 8:
            topic8[en] = topic[1]

# create a dataframe of the topic distributions
dvd_topic_dist = pd.DataFrame(
    {'Medical': topic0,
     'Surveillance': topic1,
     'Radiation': topic6,
     'Surgery': topic8
    })

In [40]:
dvd = dvd.reset_index(drop=True)
dvd_topics = pd.concat([dvd, dvd_topic_dist], axis=1)
# dvd_topics.head(20)

In [37]:
dvd_topics.to_csv('dvd_topics.csv')